# Definindo as funções

In [1]:
import cv2
import numpy as np

#Erosão
def erosion(raw_img, str_element):
    #Definindo a altura e a largura do elemento estruturante
    str_element_height = str_element.shape[0]
    str_element_width = str_element.shape[1]
    str_element_size = str_element.shape[0] #Por ser quadrado, a altura já tá bom pra definir a ordem

    #Definindo a altura e a largura da imagem a sofrer erosão
    raw_img_height = raw_img.shape[0]
    raw_img_width = raw_img.shape[1]
    
    #Criando a imagem de saída
    eroded_img = np.zeros(raw_img.shape[:2], dtype='uint8')

    #Fazendo a erosão
    for l in range(raw_img_height - (str_element_size-1)):
        for k in range(raw_img_width - (str_element_size-1)):
            flag = True
            for i in range(str_element_height):
                for j in range(str_element_width):
                    if (str_element[i,j] != raw_img[i+l,j+k]) and (str_element[i,j] == 255):
                        flag = False
                if (flag == False):
                    break
            if (flag == True):
                eroded_img[l+1,k+1] = 255
    return eroded_img

def opening(raw_img, str_element):
    #Definindo a altura e a largura do elemento estruturante
    str_element_height = str_element.shape[0]
    str_element_width = str_element.shape[1]
    str_element_size = str_element.shape[0] #Por ser quadrado, a altura já tá bom pra definir a ordem

    #Definindo a altura e a largura da imagem a sofrer abertura
    raw_img_height = raw_img.shape[0]
    raw_img_width = raw_img.shape[1]
    
    #Criando a primeira imagem de saída
    eroded_img = np.zeros(raw_img.shape[:2], dtype='uint8')

    #Fazendo a erosão
    for l in range(raw_img_height - (str_element_size-1)):
        for k in range(raw_img_width - (str_element_size-1)):
            flag = True
            for i in range(str_element_height):
                for j in range(str_element_width):
                    if (str_element[i,j] != raw_img[i+l,j+k]) and (str_element[i,j] == 255):
                        flag = False
                if (flag == False):
                    break
            if (flag == True):
                eroded_img[l+1,k+1] = 255
    
    #Criando a segunda imagem de saída
    augmented_img = np.zeros(raw_img.shape[:2], dtype='uint8')
    
    #Fazendo a dilatação
    for l in range(raw_img_height - (str_element_size-1)):
        for k in range(raw_img_width - (str_element_size-1)):
            for i in range(str_element_height):
                for j in range(str_element_width):
                    if (str_element[i,j] == eroded_img[i+l,j+k]) and (str_element[i,j] == 255):
                        augmented_img[l+1,k+1] = 255
                        break
    return augmented_img

# Fazendo a esqueletização

In [38]:
#Importando a imagem a ser esqueletizada e deixando em preto e branco
path = 'R.png'
img = cv2.imread(path, 0)
threshold, img_thresh = cv2.threshold(img, 225, 255, cv2.THRESH_BINARY_INV)

#Criando o kernel 3x3 com 1's
kernel = np.array([[  0,  0,255,  0,  0],
                   [  0,  0,255,  0,  0],
                   [255,255,255,255,255],
                   [  0,  0,255,  0,  0],
                   [  0,  0,255,  0,  0]], dtype='uint8')


kernel = np.array([[  0,255,  0],
                   [255,255,255],
                   [  0,255,  0]], dtype='uint8')

## Parte 1: A - (A opening B)

In [33]:
A = img_thresh
AoB = opening(img_thresh, kernel)

part1 = A - AoB

cv2.imshow('A', A)
cv2.imshow('AoB', AoB)
cv2.imshow('part1', part1)
cv2.waitKey()

-1

## Parte 2: (A erosion B) - ((A erosion B) opening B)

In [34]:
#(A erosion B)
A_eroded = erosion(A, kernel)

#A_eroded o B
A_eroded_then_opened = opening(A_eroded, kernel)

part2 = A_eroded - A_eroded_then_opened

cv2.imshow('part2', part2)
cv2.waitKey()

-1

## Parte 3: (A erosion 2B) - ((A erosion 2B) opening B)

In [35]:
#(A erosion 2B)
A_eroded_2 = erosion(A_eroded, kernel)

#A_eroded_2 o B
A_eroded_2_then_opened = opening(A_eroded_2, kernel)

part3 = A_eroded_2 - A_eroded_2_then_opened

cv2.imshow('part3', part3)
cv2.waitKey()

-1

In [31]:
esqueleton = part1 + part2 + part3

cv2.imshow('Original', img)
cv2.imshow('Thresh', img_thresh)
cv2.imshow('EsqueleTON', esqueleton)
cv2.waitKey()

-1

# Fazendo em formato de loop

In [41]:
A = img_thresh
B = kernel
AoB = opening(A, B)

difference = np.zeros(img.shape[:2], dtype='uint8')

for i in range(1000):
    #Fazendo a abertura de A por B, se depois da abertura não for nulo, continua
    AoB = opening(A, B)
    if (AoB.sum() == 0):
        break
    difference += (A - AoB)
    A = erosion(A,B)
    print(i)

cv2.imshow('Esq', difference)
cv2.waitKey()

0
1
2
3
4
5
6
7
8
9
10


-1

In [43]:
show = difference + img
cv2.imshow('final', show)
cv2.waitKey()

-1